## Fix symbol & Contruct big json file

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob, json

In [3]:
# Remove "null" in json files
json_file = glob.glob(pathname='*/*.json')
for j_file in json_file:
    with open(j_file, 'r', encoding='utf-8') as f:
        content = json.load(f)
        # content = json.dumps(content).replace(u'u2019', "'")
    # content_2 = json.dumps(content).replace("’", "'")
    if 'answers' in content:
        content['answers'] = [x for x in content['answers'] if x is not None]
    with open(j_file, 'w', encoding='utf-8') as f:
        json.dump(content, fp=f, indent=4, ensure_ascii=False)
    # with open(j_file, 'r', encoding='utf-8') as f:
    #     data = f.read()
    #     data = data.replace(u'\u2019', "'")
    # with open(j_file, 'w') as f:
    #     f.write(data)
    print(j_file)
    # break


Json file for web setup/auto-driving-aligned.json
Json file for web setup/bitcoin-invest-aligned.json
Json file for web setup/Would-you-get-into-a-self-driving-car_0820_1.json
Json file for web setup/Should-I-invest-in-Bitcoin_0815_1.json
big_json_data/car.json
big_json_data/bitcoin.json


### Construct big Json file

In [4]:
import os
from combine_function import percentage_and_count, create_claim_center_dict, related_questions, get_answers

In [5]:
scenario = {
    "auto-driving" : {
        "file_name" : "car.json",
        "data_base_dir" : "claim center and statistics/automos driving",
        "web_base_dir": "Json file for web setup",
        "question": "Would you get into a self-driving car?",
        "url" : "https://www.quora.com/Would-you-get-into-a-self-driving-car",
    },
    "bitcoin-invest" : {
        "file_name" : "bitcoin.json",
        "data_base_dir": "claim center and statistics/bitcoin investment",
        "web_base_dir": "Json file for web setup",
        "question" : "Should I invest in Bitcoin?",
        "url" : "https://www.quora.com/Should-I-invest-in-Bitcoin",
    }
}


In [6]:
for sc, config in scenario.items():
    # get answer
    # answer_data_path = os.path.join(config['web_base_dir'], f"{sc}-aligned.json")
    # with open(answer_data_path, 'r') as f:
    #     answer_data = json.load(f)['answers']
    answer_data = get_answers(base_dir=config['web_base_dir'], task_name=sc)
    
    statistic_data = percentage_and_count(base_dir=config['data_base_dir'])
    claim_center_data = create_claim_center_dict(base_dir=config['data_base_dir'])
    # combine data
    big_json_data = {}
    big_json_data['question'] = config['question']
    big_json_data['source'] = config['url']
    big_json_data.update(statistic_data)
    big_json_data['claim_centers'] = claim_center_data
    big_json_data['related_questions'] = related_questions(base_dir='../upvotes', task_name=sc)
    big_json_data['answers'] = answer_data

    # save big json data
    big_json_data_path = os.path.join('big_json_data', config['file_name'])
    with open(big_json_data_path, 'w') as f:
        json.dump(big_json_data, fp=f, indent=4, ensure_ascii=False)


In [7]:
## Debug
# try:
#     get_answers(base_dir=scenario['bitcoin-invest']['web_base_dir'], task_name='bitcoin-invest')
# except ValueError as err:
#     print(err)
